In [1]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.6 MB/s eta 0:00:00


In [2]:
import os
os.environ["WANDB_DISABLED"]="true"

In [3]:
import transformers
print(transformers.__version__)

4.33.0


In [13]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("DanArnin/Hinglish")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/DanArnin--Hinglish-3718595ed50128d9/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 189102
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2065
    })
})

In [15]:
raw_datasets["train"][0]

{'translation': {'en': "What's the name of the movie",
  'hi_ng': 'film ka kya naam hai',
  'source': 1}}

In [16]:
raw_datasets["validation"][0]

{'translation': {'en': ' online: http/URL',
  'hi_ng': 'snow may fall for parts of cny the next few days .  find out where the best chances will be at 6:42 am on tv &amp',
  'source': 1}}

In [17]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'play the next track', 'hi_ng': 'agle track ko chalaiye', 'source': 0}"
1,"{'en': 'What is the average humidity for the week', 'hi_ng': 'is week ke liye average humidity kya hai', 'source': 0}"
2,"{'en': 'ask julie if she can give me a ride to the airport on june 30th at 11 am', 'hi_ng': 'julie se pucho ki kya she can give me a ride to the airport on june 30th at 11 am', 'source': 0}"
3,"{'en': 'Yeah, the best film in modern animation.', 'hi_ng': 'Haan, modern animation mein best film hai.', 'source': 1}"
4,"{'en': 'How much time is left on the timer', 'hi_ng': 'timer par kitna time bacha he', 'source': 0}"


In [18]:
from transformers import AutoTokenizer

model_checkpoint = "t5-base"   
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hi_ng"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
raw_datasets['train'][:2]

{'translation': [{'en': "What's the name of the movie",
   'hi_ng': 'film ka kya naam hai',
   'source': 1},
  {'en': 'Hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. ',
   'hi_ng': 'namaste, sada hua tomatoes score mahaan hai, lekin meta critic score is gunavatta kee philm se thoda kam lagata hai.',
   'source': 1}]}

In [24]:
preprocess_function(raw_datasets['train'][:2])

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[363, 31, 7, 8, 564, 13, 8, 1974, 1], [2018, 6, 8, 3, 14369, 35, 11395, 2604, 19, 248, 68, 8, 10531, 6800, 2604, 1330, 3, 9, 385, 731, 3, 9, 1974, 13, 48, 463, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[814, 3, 1258, 3, 3781, 9, 3, 29, 9, 265, 4244, 23, 1], [3, 13363, 8323, 6, 6819, 9, 3, 107, 76, 9, 11395, 2604, 954, 1024, 152, 4244, 23, 6, 90, 2917, 10531, 6800, 2604, 19, 4740, 9, 208, 14748, 3, 1050, 15, 3, 18118, 51, 142, 3, 189, 32, 26, 9, 6511, 50, 5497, 9, 4244, 23, 5, 1]]}

In [25]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [26]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True    
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [28]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [29]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/190 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?ba/s]

In [30]:
raw_datasets['train']

Dataset({
    features: ['translation'],
    num_rows: 189102
})

In [31]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [32]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.889300,2.386338,4.077700,18.121100
2,0.718200,2.231908,4.088500,18.231000
3,0.627700,2.132591,4.268300,18.215500
4,0.598100,2.063910,4.374100,18.307500
5,0.558500,2.012233,4.594300,18.309000
6,0.537000,1.977259,4.694600,18.317200
7,0.513000,1.947072,4.917700,18.293900
8,0.523400,1.928272,5.013400,18.293900
9,0.513300,1.915793,5.030800,18.302700
10,0.507400,1.913179,4.933700,18.308000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=59100, training_loss=0.646932124736547, metrics={'train_runtime': 39492.4112, 'train_samples_per_second': 47.883, 'train_steps_per_second': 1.496, 'total_flos': 7.781055072049152e+16, 'train_loss': 0.646932124736547, 'epoch': 10.0})

In [33]:
print("hello")

hello


In [34]:
!zip -r file.zip /kaggle/working

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/ (stored 0%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/runs/ (stored 0%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/runs/Sep27_15-28-46_43d49d8280d2/ (stored 0%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/runs/Sep27_15-28-46_43d49d8280d2/events.out.tfevents.1695828559.43d49d8280d2.28.0 (deflated 66%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/ (stored 0%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/special_tokens_map.json (deflated 86%)
  adding: kaggle/working/t5-base-finetuned-en-to

In [41]:
!zip -r file2.zip /kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/ (stored 0%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/special_tokens_map.json (deflated 86%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/tokenizer.json (deflated 74%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/spiece.model (deflated 48%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/tokenizer_config.json (deflated 83%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000/config.json (deflated 62%)
  adding: kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-5900

In [42]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
file2.zip  t5-base-finetuned-en-to-hi_ng


In [43]:
from IPython.display import FileLink
FileLink(r'file2.zip')

/kaggle/working/file2.zip

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working/t5-small-finetuned-en-to-hi_ng/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [38]:
from transformers import  T5Tokenizer,T5ForConditionalGeneration


model_name = '/kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000'
tokenizer = T5Tokenizer.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [52]:
src_text = ['Batman and robin have joker as their enemy,']

In [76]:
model = T5ForConditionalGeneration.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
val = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [77]:
val

['Batman aur robin ko joker ka enemy hai,']

In [1]:
pip install englisttohindi

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=df51d01d6a32a9b75f38bcdd6bfbd790d88b9f0a4040b12e163c61631eec3f0b
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [95]:

 
# message to be translated
message = "Yes"
 
# creating a EngtoHindi() object
res = EngtoHindi(message)
 
# displaying the translation
print(res.convert)

हाँ


In [96]:
res.convert

'हाँ'

In [70]:
pip install pyenchant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [3]:
!ls

In [6]:
!cd /kaggle/working/t5-base-finetuned-en-to-hi_ng/

/bin/bash: line 1: cd: /kaggle/working/t5-base-finetuned-en-to-hi_ng/: No such file or directory


In [5]:
!ls

In [2]:
import nltk
from englisttohindi.englisttohindi import EngtoHindi
from transformers import  T5Tokenizer,T5ForConditionalGeneration
model_name = '/kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000'
tokenizer = T5Tokenizer.from_pretrained(model_name)
nltk.download('words')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/working/t5-base-finetuned-en-to-hi_ng/checkpoint-59000'. Use `repo_type` argument if needed.

In [91]:
def check_word(word):
    return word in nltk.corpus.words.words()

In [ ]:
def hindi_replacement(translated_string):
    final_translation = ''
    print(type(translated_string))
    for i in translated_string:
        if check_word(i):
            final_translation += i
        else:
            print(i)
#             temp = EngtoHindi(i)
#             final_translation += temp.convert
    return final_translation
            

In [ ]:
def translator(src_text):
    final_val = []
    translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
    val = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    for i in val:
        temp = hindi_replacement(i)
        final_val.append(temp)
    return final_val
        
    

text = ['Hello my name is Dharshan' , 'I like cycling on rainy days', 'Batman and robins enemy is joker']
translator(text)

In [ ]:
text = ['Hello my name is Dharshan' , 'I like cycling on rainy days', 'Batman and robins enemy is joker'] 
translator(text)